# This note book helps to merge indiviual mdoc files (one per movie) to a single mdoc file (one per tilt-series)
it also has a section to check for duplicate aquistions (at same tilt angle)

- create conda env:
- conda create -n teamtomo
- conda activate teamtomo
- pip install mdocfile
- conda install  jupyter

In [1]:
import mdocfile
import pandas as pd
import os
from mdocfile.data_models import Mdoc, MdocGlobalData, MdocSectionData
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)


In [2]:
print(os.getcwd())

/mnt/slinky-001/sconnell/Projects/IGBMC-0001-RsmATomo/DATA/DATA/2024-04-03_Src5-pos3


#### Here we find the files that have the common identifier = first x character and place in a dictionary of lists (key = identifier and item = list of file name)

In [7]:
cut = 15 # cut file name after x characters = identifier # edit here

import glob
filelist = glob.glob('Src5*.mdoc') # edit here

tseries = {}  
for x in filelist: 
    #print(x[:15])
    key = x[:cut] # The key is the first 'cut' characters of the file name
    group = tseries.get(key,[])
    group.append(x)  
    tseries[key] = group

sorted_tseries = dict(sorted(tseries.items()))
print(len(sorted_tseries.keys()))
print(sorted_tseries.keys())

24
dict_keys(['Src5_pos3_ts-67', 'Src5_pos3_ts-68', 'Src5_pos3_ts-69', 'Src5_pos3_ts-70', 'Src5_pos3_ts-71', 'Src5_pos3_ts-72', 'Src5_pos3_ts-73', 'Src5_pos3_ts-74', 'Src5_pos3_ts-75', 'Src5_pos3_ts-76', 'Src5_pos3_ts-77', 'Src5_pos3_ts-78', 'Src5_pos3_ts-79', 'Src5_pos3_ts-80', 'Src5_pos3_ts-81', 'Src5_pos3_ts-82', 'Src5_pos3_ts-83', 'Src5_pos3_ts-84', 'Src5_pos3_ts-85', 'Src5_pos3_ts-86', 'Src5_pos3_ts-87', 'Src5_pos3_ts-88', 'Src5_pos3_ts-89', 'Src5_pos3_ts-90'])


#### Here we find TS (by identifier) that have duplications of acquisition angles

In [8]:
for tsname, flist in sorted_tseries.items():
    # edit splice so you cut from beginning and end to get the angle
    t=[float(w[22:-24]) for w in flist] # edit here
    if len(t) != len(set(t)):
        dup = {x for x in t if t.count(x) > 1} # this is set
        print(tsname, len(flist), len(t), len(set(t)), 'Duplication', dup)
    else:
        print(tsname, len(flist), len(t), len(set(t)))

Src5_pos3_ts-67 71 71 56 Duplication {2.0, 34.0, 6.0, 38.0, 8.0, 10.0, -22.0, 12.0, 60.0, 14.0, 16.0, -16.0, -4.0}
Src5_pos3_ts-68 57 57 56 Duplication {-30.0}
Src5_pos3_ts-69 44 44 34 Duplication {-24.0, -22.0, -16.0, -14.0, 18.0, -10.0, -6.0}
Src5_pos3_ts-70 51 51 48 Duplication {56.0}
Src5_pos3_ts-71 60 60 60
Src5_pos3_ts-72 60 60 60
Src5_pos3_ts-73 23 23 22 Duplication {-4.0}
Src5_pos3_ts-74 60 60 60
Src5_pos3_ts-75 58 58 58
Src5_pos3_ts-76 60 60 60
Src5_pos3_ts-77 60 60 60
Src5_pos3_ts-78 59 59 59
Src5_pos3_ts-79 60 60 60
Src5_pos3_ts-80 60 60 60
Src5_pos3_ts-81 60 60 60
Src5_pos3_ts-82 60 60 60
Src5_pos3_ts-83 60 60 60
Src5_pos3_ts-84 57 57 57
Src5_pos3_ts-85 61 61 60 Duplication {32.0}
Src5_pos3_ts-86 60 60 60
Src5_pos3_ts-87 60 60 60
Src5_pos3_ts-88 58 58 58
Src5_pos3_ts-89 53 53 53
Src5_pos3_ts-90 63 63 59 Duplication {64.0, 60.0}


#### Here we use mdocfile module to interate over the indiviudal mdocs, extract the section data, update the Zvalue with the index (order of aquistion) and write out a merged mdoc file.

In [9]:
for tsname, flist in sorted_tseries.items():
    mdoc_section_data = [] # initialize a empty list for section_data
    #print(tsname, len(flist))
    #sort this list based on the index in the file name
    files=sorted(flist, key = lambda x: x[16:21]) # edit here
    #construct output file name
    fout = f'{files[0][10:15]}.mdoc'
    #print(fout)
    # read in first file to construct global data
    mdoc = Mdoc.from_file(files[0])
    global_data = mdoc.global_data 

    # construct section data models and update FrameSet value with index
    for f in files:
        section_data = Mdoc.from_file(f).section_data[0] # here input mdocs only have one section
        #print(section_data.FrameSet)
        section_data.ZValue=int(f[16:21]) # these characters that mark the index in the file name 
        mdoc_section_data.append(section_data)
        #print(section_data.FrameSet)

    # construct mdoc model
    mdoc = Mdoc(
        titles=[
        '[T = SerialEM: Krios G4 9930510 at IGBMC   ]',
        ],
        global_data=global_data,
        section_data=mdoc_section_data
    )

    # write out the file
    print(f'Writing merged mdoc file:    {fout}')
    with open(fout, mode='w+') as file:
        file.write(mdoc.to_string())

Writing merged mdoc file:    ts-67.mdoc
Writing merged mdoc file:    ts-68.mdoc
Writing merged mdoc file:    ts-69.mdoc
Writing merged mdoc file:    ts-70.mdoc
Writing merged mdoc file:    ts-71.mdoc
Writing merged mdoc file:    ts-72.mdoc
Writing merged mdoc file:    ts-73.mdoc
Writing merged mdoc file:    ts-74.mdoc
Writing merged mdoc file:    ts-75.mdoc
Writing merged mdoc file:    ts-76.mdoc
Writing merged mdoc file:    ts-77.mdoc
Writing merged mdoc file:    ts-78.mdoc
Writing merged mdoc file:    ts-79.mdoc
Writing merged mdoc file:    ts-80.mdoc
Writing merged mdoc file:    ts-81.mdoc
Writing merged mdoc file:    ts-82.mdoc
Writing merged mdoc file:    ts-83.mdoc
Writing merged mdoc file:    ts-84.mdoc
Writing merged mdoc file:    ts-85.mdoc
Writing merged mdoc file:    ts-86.mdoc
Writing merged mdoc file:    ts-87.mdoc
Writing merged mdoc file:    ts-88.mdoc
Writing merged mdoc file:    ts-89.mdoc
Writing merged mdoc file:    ts-90.mdoc
